### Day 1 (chapters 1 & 2 from Spark:The definitive guide)

1. **What is bigdata?**

    The definition of big data is data that contains greater variety, arriving in increasing volumes and with more velocity. This is also known as the three Vs. Put simply, big data is larger, more complex data sets, especially from new data sources. These data sets are so voluminous that traditional data processing software just can’t manage them. But these massive volumes of data can be used to address business problems you wouldn’t have been able to tackle before. 
     
2. **Why spark?**

    Cluster computing holds tre‐mendous potential. At every organization that uses MapReduce, brand new applications could be built using the existing data. However, the MapReduce engine made it both challenging and inefficient to build large applications. To address this problem, the Spark team first designed an API based on functional programming that could succinctly express multistep applications. This new engine could also perform efficient, in-memory data sharing across computation steps. 

3. **What is spark?**

    Apache Spark is an open-source, distributed processing system used for big data workloads. It utilizes in-memory caching and optimized query execution for fast queries against data of any size. Simply put, Spark is a fast and general engine for large-scale data processing.

4. **Internals of spark?**

    Spark Applications consist of a **driver** process and a set of **executor** processes. 
    
    The **driver** process runs your main() function, sits on a node in the cluster, and is responsible for three things: maintaining information about the Spark Application; responding to a user’s program or input; and analyzing, distributing, and scheduling work across the executors (discussed momentarily). The driver process is absolutely essential it’s the heart of a Spark Application and maintains all relevant information during the lifetime of the application. 
    
    The **executors** are responsible for actually carrying out the work that the driver assigns them. This means that each executor is responsible for only two things: executing code assigned to it by the driver, and reporting the state of the computation on that executor back to the driver node.

5. **Highlevel API of spark?**

    - **Sparksession**: We can control our Spark Application through a driver process called the SparkSession. The SparkSession instance is the way Spark executes user-defined manipulations across the cluster.
    - **Dataframe**: A DataFrame is the most common Structured API and simply represents a table of data with rows and columns. The list that defines the columns and the types within those columns is called the schema. 
    - **Partitions**: To allow every executor to perform work in parallel, Spark breaks up the data into chunks called partitions. A partition is a collection of rows that sit on one physical machine in your cluster. 
    - **Transformation**: To “change” a DataFrame, you need to instruct Spark how you would like to modify it to do what you want. These instructions are called transformations.
    - **Actions**: Transformations allow us to build up our logical transformation plan. To trigger the computation, we run an action. An action instructs Spark to compute a result from a series of transformations. 
    - **Lazy Evaluation**: Lazy evaulation means that Spark will wait until the very last moment to execute the graph of computation instructions. In Spark, instead of modifying the data immediately when you express some operation, you build up a plan of transformations that you would like to apply to your source data.

### Day 2 (chapter 4, 5)

1. **Structured API**
    - **SQL**: One use of Spark SQL is to execute SQL queries. Spark SQL can also be used to read data from an existing Hive installation. When running SQL from within another programming language the results will be returned as a Dataset/DataFrame which I will elaborate below.
    - **Dataset**: A Dataset is a distributed collection of data. Dataset provides the benefits of RDDs (strong typing, ability to use powerful lambda functions) with the benefits of Spark SQL’s optimized execution engine. A Dataset can be constructed from JVM objects and then manipulated using functional transformations (map, flatMap, filter, etc.).
    - **Dataframes**: A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. 

2. **Basic Structured Operation**

1. **Schemas**

In [0]:
spark.read.format("delta").load("dbfs:/user/hive/warehouse/ak_2015_summary").schema

Out[1]: StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])

2. **Columns and Expressions**

In [0]:
from pyspark.sql.functions import col, column
col("count")
column("count")

(((col("someCol") + 5) * 200) - 6) < col("otherCol")

from pyspark.sql.functions import expr
expr("(((someCol + 5) * 200) - 6) < otherCol")

spark.read.format("delta").load("dbfs:/user/hive/warehouse/ak_2015_summary").columns

Out[6]: ['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

3. **Records and Rows**

In [0]:
df = spark.read.format("delta").load("dbfs:/user/hive/warehouse/ak_2015_summary")
df.first()

Out[7]: Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

4. **Create Rows**

In [0]:
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

myRow[0]
myRow[2]

Out[9]: 1

5. **Dataframe Transformation**

Several types of transformations are possible which include:
- Adding columns or rows
- Removing rows or columns
- Transforming a row into a column and vice versa
- Changing the order of rows based on values in columns

6. **Creating Dataframes**

In [0]:
df = spark.read.format("delta").load("dbfs:/user/hive/warehouse/ak_2015_summary")
df.createOrReplaceTempView("dfTable")

from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType
myManualSchema = StructType([
 StructField("some", StringType(), True),
 StructField("col", StringType(), True),
 StructField("names", LongType(), False)
])
myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



7. **select and selectExpr**

In [0]:
df = spark.read.load("dbfs:/user/hive/warehouse/ak_2015_summary")
df.select("DEST_COUNTRY_NAME").show(2)

df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

from pyspark.sql.functions import expr, col, column
df.select(
 expr("DEST_COUNTRY_NAME"),
 col("DEST_COUNTRY_NAME"),
 column("DEST_COUNTRY_NAME"))\
 .show(2)

df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME"))\
 .show(2)

df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

df.selectExpr(
 "*", # all original columns
 "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
 .show(2)

df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

from pyspark.sql.functions import lit
df.select(expr("*"), lit(1).alias("One")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-------

8. **Adding columns**

In [0]:
df.withColumn("numberOne", lit(1)).show(2)

df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME"))\
 .show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



9. **Renaming columns**

In [0]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

Out[35]: ['dest', 'ORIGIN_COUNTRY_NAME', 'count']

10. **Changing column's type**

In [0]:
df.withColumn("count2", col("count").cast("long"))

Out[36]: DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: bigint]

11. **Filtering rows**

In [0]:
df.filter(col("count") < 2).show(2)
df.where("count < 2").show(2)

df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia")\
 .show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



12. **Getting unique rows**

In [0]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

df.select("ORIGIN_COUNTRY_NAME").distinct().count()

Out[40]: 125

13. **Sorting**

In [0]:
df.sort("count").show(5)
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

from pyspark.sql.functions import desc, asc
df.orderBy(expr("count desc")).show(2)
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)

spark.read.format("delta").load("dbfs:/user/hive/warehouse/ak_2015_summary")\
 .sortWithinPartitions("count")

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--

14. **Limit**

In [0]:
df.limit(5).show()

df.orderBy(expr("count desc")).limit(6).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
+--------------------+-------------------+-----+



15. **Repartition and Coalesce**

In [0]:
df.rdd.getNumPartitions() # 1

df.repartition(5)

df.repartition(col("DEST_COUNTRY_NAME"))

df.repartition(5, col("DEST_COUNTRY_NAME"))

df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

Out[51]: DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

### Day 3 (chapter 7)

1. **Aggregation Functions**

1. **count**

In [0]:
df = spark.read.format("delta")\
 .option("header", "true")\
 .option("inferSchema", "true")\
 .load("dbfs:/user/hive/warehouse/online_retail_dataset")\
 .coalesce(5)
df.cache()
df.createOrReplaceTempView("dfTable")

df.count() == 541909

from pyspark.sql.functions import count
df.select(count("StockCode")).show() # 541909

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



2. **countDistinct**

In [0]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct("StockCode")).show() # 4070

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



3. **first and last**

In [0]:
from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



4. **min and max**

In [0]:
from pyspark.sql.functions import min, max
df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



5. **sum**

In [0]:
from pyspark.sql.functions import sum
df.select(sum("Quantity")).show() # 5176450

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



6. **sumDistinct**

In [0]:
from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("Quantity")).show() # 29310

/databricks/spark/python/pyspark/sql/functions.py:723: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



7. **avg**

In [0]:
from pyspark.sql.functions import sum, count, avg, expr
df.select(
 count("Quantity").alias("total_transactions"),
 sum("Quantity").alias("total_purchases"),
 avg("Quantity").alias("avg_purchases"),
 expr("mean(Quantity)").alias("mean_purchases"))\
 .selectExpr(
 "total_purchases/total_transactions",
 "avg_purchases",
 "mean_purchases").show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



8. **grouping**

In [0]:
df.groupBy("InvoiceNo", "CustomerId").count().show()

from pyspark.sql.functions import count
df.groupBy("InvoiceNo").agg(
 count("Quantity").alias("quan"),
 expr("count(Quantity)")).show()

df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)"))\
 .show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536415|     12838|   59|
|   536748|     15107|    5|
|   536788|     15061|    1|
|   537694|     14901|   14|
|   537796|     18061|    6|
|   537833|     13270|    1|
|   538872|     13097|   11|
|   539326|     14560|   14|
|  C539486|     18256|    4|
|   540354|     13576|   16|
|   540371|     14312|   15|
|  C541222|     14299|    1|
|   541265|     17609|  100|
|   541847|     15443|   51|
|   542542|     12431|   20|
|   543009|     18041|   10|
|  C543290|     16686|    2|
|   543459|     12599|   14|
|   543550|     12601|    6|
|   543808|     17454|   29|
+---------+----------+-----+
only showing top 20 rows

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
|   537252|   1|              1|
|   537691|  20|             20|
|   538041|   1|              1|
|   538184|  2

9. **window functions**

In [0]:
from pyspark.sql.functions import col, to_date
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

from pyspark.sql.window import Window
from pyspark.sql.functions import desc
windowSpec = Window\
 .partitionBy("CustomerId", "date")\
 .orderBy(desc("Quantity"))\
 .rowsBetween(Window.unboundedPreceding, Window.currentRow)

from pyspark.sql.functions import max
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

from pyspark.sql.functions import dense_rank, rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
from pyspark.sql.functions import col
dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
 .select(
 col("CustomerId"),
 col("date"),
 col("Quantity"),
 purchaseRank.alias("quantityRank"),
 purchaseDenseRank.alias("quantityDenseRank"),
 maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

### Day 4 (chapter 8)

1. **Joins**

1. **Try out examples for each types of joins**

In [0]:
person = spark.createDataFrame([
 (0, "Bill Chambers", 0, [100]),
 (1, "Matei Zaharia", 1, [500, 250, 100]),
 (2, "Michael Armbrust", 1, [250, 100])])\
 .toDF("id", "name", "graduate_program", "spark_status")
graduateProgram = spark.createDataFrame([
 (0, "Masters", "School of Information", "UC Berkeley"),
 (2, "Masters", "EECS", "UC Berkeley"),
 (1, "Ph.D.", "EECS", "UC Berkeley")])\
 .toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
 (500, "Vice President"),
 (250, "PMC Member"),
 (100, "Contributor")])\
 .toDF("id", "status")

person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [0]:
# Inner Joins
joinExpression = person["graduate_program"] == graduateProgram['id']

wrongJoinExpression = person["name"] == graduateProgram["school"]

person.join(graduateProgram, joinExpression).show()

joinType = "inner"
person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+--

In [0]:
# Outer Joins
joinType = "outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [0]:
# Left Outer Joins
joinType = "left_outer"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   0|   Bill Chambers|               0|          [100]|
|  2|Masters|                EECS|UC Berkeley|null|            null|            null|           null|
|  1|  Ph.D.|                EECS|UC Berkeley|   2|Michael Armbrust|               1|     [250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|   1|   Matei Zaharia|               1|[500, 250, 100]|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+



In [0]:
# Right Outer Joins
joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [0]:
# Left Semi Joins
joinType = "left_semi"
graduateProgram.join(person, joinExpression, joinType).show()

gradProgram2 = graduateProgram.union(spark.createDataFrame([
 (0, "Masters", "Duplicated Row", "Duplicated School")]))
gradProgram2.createOrReplaceTempView("gradProgram2")
gradProgram2.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  1|  Ph.D.|                EECS|      UC Berkeley|
|  0|Masters|      Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+



In [0]:
# Left Anti Joins
joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



In [0]:
# Cross (Cartersian) Joins
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()

person.crossJoin(graduateProgram).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+--

2. **Handling Duplicate column names**

In [0]:
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")
joinExpr =  gradProgramDupe.select("graduate_program")

person.join(gradProgramDupe,"graduate_program").select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



3. **How spark performs joins**

    - **Big table-to-big table**: When you join a big table to another big table, you end up with a shuffle join. In a shuffle join, every node talks to every other node and they share data according to which node has a certain key or set of keys (on which you are joining). These joins are expensive because the network can become congested with traffic, especially if your data is not partitioned well.
    - **Big table-to-small table**: When the table is small enough to fit into the memory of a single worker node, with some breathing room of course, we can optimize our join. Here we will replicate our small DataFrame onto every worker node in the cluster (be it located on one machine or many). This does prevent us from performing the all-to-all communication during the entire join process. This means that joins will be performed on every single node individually, making CPU the biggest bottleneck.
    - **Little table-to-little table**: When performing joins with small tables, it’s usually best to let Spark decide how to join them. You can always force a broadcast join if you’re noticing strange behavior

### Day 5 (chapter 9)

1. **Datasources**

1. **Basics of reading data**

The foundation for reading data in Spark is the *DataFrameReader*. We access this through the SparkSession via the read attribute: \
*spark.read*

Here’s an example of the overall layout: \
*spark.read.format("csv") \
 .option("mode", "FAILFAST") \
 .option("inferSchema", "true") \
 .option("path", "path/to/file(s)") \
 .schema(someSchema) \
 .load()*

The default read mode is *permissive*

2. **Basics of write data**

The core structure for writing data is as follows: \
*DataFrameWriter.format(...).option(...).partitionBy(...).bucketBy(...).sortBy(...).save()*

Example: \
*dataframe.write.format("csv") \
 .option("mode", "OVERWRITE") \
 .option("dateFormat", "yyyy-MM-dd") \
 .option("path", "path/to/file(s)") \
 .save()*

The default write mode is *errorIfExists*

3. **CSV files - reading, writing**

In [0]:
# Reading a csv file
csvFile = spark.read.format("delta")\
 .option("header", "true")\
 .option("mode", "FAILFAST")\
 .option("inferSchema", "true")\
 .load("dbfs:/user/hive/warehouse/online_retail_dataset")

# Writing a csv file
csvFile.write.format("csv").mode("overwrite").option("sep", "\t")\
 .save("/tmp/my-tsv-file.tsv")

4. **REading and writing json files**

In [0]:
# Reading a JSON file
spark.read.format("delta").option("mode", "FAILFAST")\
 .option("inferSchema", "true")\
 .load("dbfs:/user/hive/warehouse/ak_2015_summary").show(5)

# Writing a JSON file
csvFile.write.format("json").mode("overwrite").save("/tmp/my-json-file.json")

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



5. **Parquet files - important**

Parquet is an open source column-oriented data store that provides a variety of storage optimizations, especially for analytics workloads. It provides columnar compression, which saves storage space and allows for reading individual columns instead of
entire files. It is a file format that works exceptionally well with Apache Spark and is in fact the default file format.
Here’s how to specify Parquet as the read format: \
*spark.read.format("parquet")*

6. **Reading and Writing parquet files**

*Note: Databricks doesn't support uploading parquet files as a dataset option so moving ahead with JSON file option only*

In [0]:
# Reading a file
spark.read.format("delta") \
.load("dbfs:/user/hive/warehouse/ak_2015_summary").show(5)

# Writing a file
csvFile.write.format("parquet").mode("overwrite")\
 .save("/tmp/my-parquet-file.parquet")


+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



7. **orc - optional**

*Note: Databricks doesn't support uploading ORC files as a dataset option so skipping this as it is optional*

8. **Splittable File Types and COmpression**

Certain file formats are fundamentally “splittable.” This can improve speed because it makes it possible for Spark to avoid reading an entire file, and access only the parts of the file necessary to satisfy your query. \
In conjunction with this is a need to manage compression. Not all compression schemes are splittable. How you store your data is
of immense consequence when it comes to making your Spark jobs run smoothly.

9. **Managing File size**

Managing file sizes is an important factor not so much for writing data but reading it later on. Spark especially does not do well with
small files, although many file systems (like HDFS) don’t handle lots of small files well, either. You might hear this referred to as the “small file problem.” \
Spark 2.2 introduced a new method for controlling file sizes in a more automatic way. You can use the **maxRecordsPerFile** option and specify a number of your choosing.  For example, if you set an option for a writer as **df.write.option("maxRecordsPerFile", 5000)**, Spark will ensure that files will contain at most 5,000 records.

### Day 6 (chapter 10)

**Entire chapter of Spark SQL**

**Spark’s Programmatic SQL Interface**

In [0]:
spark.sql("SELECT 1 + 1").show()

spark.read.format("delta").load("dbfs:/user/hive/warehouse/ak_2015_summary")\
 .createOrReplaceTempView("some_sql_view") # DF => SQL
spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count)
FROM some_sql_view GROUP BY DEST_COUNTRY_NAME
""")\
 .where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10")\
 .count() # SQL => DF

+-------+
|(1 + 1)|
+-------+
|      2|
+-------+

Out[134]: 12

**Creating Tables**

In [0]:
spark.sql("""
          CREATE TABLE flights (
 DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
USING JSON OPTIONS (path 'dbfs:/user/hive/warehouse/ak_2015_summary')""")

Out[4]: DataFrame[]

In [0]:
spark.sql("""
          CREATE TABLE flights_csv (
 DEST_COUNTRY_NAME STRING,
 ORIGIN_COUNTRY_NAME STRING COMMENT "remember, the US will be most prevalent",
 count LONG)
USING csv OPTIONS (header true, path 'dbfs:/user/hive/warehouse/ak_2015_summary')""")

Out[142]: DataFrame[]

In [0]:
spark.sql("""SET spark.databricks.delta.formatCheck.enabled=false""")
spark.sql("""CREATE TABLE flights_from_select AS SELECT * FROM flights""")

Out[8]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("""CREATE TABLE IF NOT EXISTS flights_from_select
 AS SELECT * FROM flights""")

Out[9]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("""CREATE TABLE partitioned_flights USING parquet PARTITIONED BY (DEST_COUNTRY_NAME)
AS SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 5""")

Out[10]: DataFrame[]

**Creating External Tables**

In [0]:
spark.sql("""
          CREATE EXTERNAL TABLE hive_flights (
 DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION 'dbfs:/user/hive/warehouse/ak_2015_summary'
""")

Out[146]: DataFrame[]

In [0]:
spark.sql("""CREATE EXTERNAL TABLE hive_flights_2
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LOCATION '/data/flight-data-hive/' AS SELECT * FROM flights""")

Out[11]: DataFrame[]

**Inserting into Tables**

In [0]:
spark.sql("""INSERT INTO flights_from_select
 SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 20""")

Out[12]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

**Describing Table Metadata**

In [0]:
spark.sql("""DESCRIBE TABLE flights_csv""")

Out[14]: DataFrame[col_name: string, data_type: string, comment: string]

In [0]:
spark.sql("""SHOW PARTITIONS partitioned_flights""")

Out[15]: DataFrame[partition: string]

**Refreshing Table Metadata**

In [0]:
spark.sql("""REFRESH table partitioned_flights""")

Out[16]: DataFrame[]

In [0]:
spark.sql("""MSCK REPAIR TABLE partitioned_flights""")

Out[17]: DataFrame[]

**Dropping Tables**

In [0]:
spark.sql("""DROP TABLE flights_csv""")

Out[18]: DataFrame[]

In [0]:
spark.sql("""DROP TABLE IF EXISTS flights_csv""")

Out[19]: DataFrame[]

**Caching Tables**

In [0]:
spark.sql("""CACHE TABLE flights""")
spark.sql("""UNCACHE TABLE FLIGHTS""")

Out[20]: DataFrame[]

**Creating Views**

In [0]:
spark.sql("""CREATE VIEW just_usa_view AS
 SELECT * FROM flights WHERE dest_country_name = 'United States'
""")

spark.sql("""CREATE TEMP VIEW just_usa_view_temp AS
 SELECT * FROM flights WHERE dest_country_name = 'United States'
""")

spark.sql("""CREATE GLOBAL TEMP VIEW just_usa_global_view_temp AS
 SELECT * FROM flights WHERE dest_country_name = 'United States'
""")

spark.sql("""SHOW TABLES
""")

Out[21]: DataFrame[database: string, tableName: string, isTemporary: boolean]

In [0]:
spark.sql("""CREATE OR REPLACE TEMP VIEW just_usa_view_temp AS
 SELECT * FROM flights WHERE dest_country_name = 'United States'""")

spark.sql("""SELECT * FROM just_usa_view_temp""")

Out[23]: DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [0]:
spark.sql("""EXPLAIN SELECT * FROM just_usa_view""")

spark.sql("""EXPLAIN SELECT * FROM flights WHERE dest_country_name = 'United States'""")

Out[24]: DataFrame[plan: string]

**Dropping Views**

In [0]:
spark.sql("""DROP VIEW IF EXISTS just_usa_view""")

Out[25]: DataFrame[]

**Databases**

In [0]:
spark.sql("""SHOW DATABASES""")

Out[26]: DataFrame[databaseName: string]

**Creating Databases**

In [0]:
spark.sql("""CREATE DATABASE some_db""")

Out[27]: DataFrame[]

**Setting the Database**

In [0]:
spark.sql("""USE some_db""")

spark.sql("""SHOW tables""")

spark.sql("""SELECT * FROM default.flights""")

Out[29]: DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [0]:
spark.sql("""SELECT current_database()""")

spark.sql("""USE default""")

Out[30]: DataFrame[]

**Dropping Databases**

In [0]:
spark.sql("""DROP DATABASE IF EXISTS some_db""")

Out[31]: DataFrame[]